<a href="https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/CRFs/CRFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional Random Fields

條件隨機場是一個統計應用於神經網路的方法，他可以用於影像分割、自然語言處理等等，通常會接在網路後端對影像或者文字做進一步的優化，這裡我們著重介紹 CRFs 在影像分割上的用途

## From Log Linear Model to CRFs

我們回憶我們學到的 log linear model 定義為 

$$p(y|x)=\frac{1}{Z(x)}\exp\sum_{k=1}^Kw_kF_k(x,y)$$

- $Z(x)=\sum_y\exp\sum_{k=1}^Kw_kF_k(x,y)$ 被稱作標準化因子 (normalized factor)，其作用保證機率值介於 $[0,1]$ 區間，且在同一個 $x$ 下有 $\sum_xp(y|x)=1$ 

- $F(x, y)$ 稱作特徵函數 (feature function)，用以衡量 $x$ 和 $y$ 共同出現的適配程度

CRFs 可以看做是 Log Linear Model 的一種，他把特徵函數定義為 $F(x,y)=\sum_{i=2}^nf_k(y_{i-1}, y_i,x,i)$

## From HMM to CRFs

另一種代入 CRFs 的看法可以從 HMM (Hiden Markov Model) 講起，HMM 是個生成模型且定義如下

$$p(x,y)=p(y)p(x|y)=\prod_{t=1}^Tp(y_t|y_{t-1})p(x_t|y_t)$$

- $x_t$ 為觀測值，$y_t$ 為隱藏態 (hidden state)
- 馬可夫假設當前狀態 (current state) $y_t$ 只被前狀態 (previous state) 影響 $y_{t-1}$
- $p(y_t|y_{t-1})$ 被稱作轉移機率 (transition probability)，表示狀態之間轉移的可能性
- $p(x_t|y_t)$ 被稱作排放機率 (emission probability)，表因當前狀態而產生 (排出) 觀測值 $x_t$ 的可能性

不難看出，上式第一個等號為條件機率，其中 $y=\{y_1,\cdots, y_t\}$ 為狀態序列，第二個等號表示從第一個狀態轉移到最後一個狀態的機率，乘上狀態產生觀測值的機率


![https://i.stack.imgur.com/khcnl.png](https://i.stack.imgur.com/khcnl.png)

## CRFs

我們回頭看看 CRFs 的特徵函數

- $p(y|x)=\frac{1}{Z(x)}\exp\sum_k w_k\sum_i f_k(y_{i-1}, y_i,x,i)$
- $\sum_i f_k(y_{i-1}, y_i,x,i)$ 代表在馬可夫假設下的任意子結構, 更精確的說, 給定一個觀測值 $x$, 他可以與任意隱狀態 $y_i$ 相關, 且馬可夫自然條件假設 $y_i$ 與 $y_{i-1}$ 相關
- Linear-chain CRFs 允許觀測值之間的任意連接
- $i$ 是一個強力參數，可用來考慮狀態的時間位置
- $w_k$ 用以衡量特徵函數的重要性，可以透過梯度下降學習

## CRFs In CV

我們來講講在電腦視覺領域上的 CRFs 如何建模，回憶我們前文所提，我們意旨於找到一個隱藏序列可以最大化給定觀測值的機率，即

$$
\arg\max_y p(y|x)
$$

在影像分割領域中

- Consider an image $I$ with $n$ pixels segmentate to $k$ classes
- Model the segmentation to a random field $X=\{X_1,\cdots, X_n\}$ where $X_i$ takes value in $1,\cdots,k$
- $y:$ 影像分割的 Output
- $x:$ 原影像
- Let $C$ be a clique, $X_C$ be the observation data in $C$
- $p(X)=\frac{1}{Z}\prod_C\psi_C(X_C)$ where potential function is defined as $\psi_C(X_C)=e^{-E(X_C)}$ and $E(X_C)=\sum_i\psi_u(x_i)+\sum_{i<j}\psi_p(x_i, x_j)$

### Energy function

- $E(X_C)=\sum_i\psi_u(x_i)+\sum_{i<j}\psi_p(x_i, x_j)$
- The value of energy function indicates that the degree of stationary of the state (lower value indicate stationary)
- $\sum_i\psi_u(x_i):$ the pixels assign the wrong labels $(|X_C|=1)$
- $\sum_{i<j}\psi_p(x_i, x_j):$ all the pair of points assign different labels $(|X_C|=2)$

注意到在能量函數中，unary potential 通常為 CNN 模型的輸出，pairwise potential 則是需要去決定的函數

### MF Inference

![](https://i.imgur.com/PHFHzHk.jpg)

## 直覺上怎麼說?

看了這麼多數學，都快吐了，我們來看看直覺上 CRFs 代給我們什麼?

![](https://i.imgur.com/uwUVOtS.png)

- (a) 原影像
- (b) 將影像的 pixel 點建模成隨機場的形式
- (c) 通過 CNN 得到 unary output (這時有些點被分錯) (unary potential)

<details>
<summary>為什麼會分錯呢? 點擊查看解答</summary>
CNN 只關注 local 範圍的物件
</details>

- (d) 這時候我們可以根據影像的一些特徵，如 pixel 之間的亮度，位置等等去懲罰那些分錯的點 (pairwise potential)

### FullCRFs

- $\psi_p(x_i,x_j|I)=\mu(x_i,x_j)\sum_{m=1}^Mw^{(m)}k^{(w)}_G(f_i^I,f_j^I)$
- $w^{(m)}$ are the learnable parameters of model
- $k(f_i^I,f_j^I)=\\w^{(1)}\exp(-\frac{|p_i-p_j|^2}{2\theta_\alpha^2}-\frac{|I_i-I_j|^2}{2\theta_\beta^2})+w^{(2)}\exp(-\frac{|p_i-p_j|^2}{2\theta_\gamma^2})$
- $w^{(1)}, w^{(2)},\theta_\alpha, \theta_\beta, \theta_\gamma$ are learnable parameters
- ConvCRFs complains that the message passing is the bottleneck of CRF with $O(\mbox{number of pixels}^2)$ time complexity

### ConvCRFs

- Consider an image $I$ with $n$ pixels segmentate to $k$ classes
- Model the segmentation to a random field $X=\{X_1,\cdots, X_n\}$ where $X_i$ takes value in $1,\cdots,k$
- Goal: solve $\arg\max_XP(X|I)$
- $P(X|I)$ is modeled as $P(X=\hat x|I=\tilde I)=\frac{1}{Z(I)}\exp(-E(\hat x|I))$ where $E(\hat x|I)=\sum_{}\psi_u(\hat x_i|I)+\sum_{i\neq j\leq N}\psi_p(\hat x_i,\hat x_j|I)$

### What's New?

- ConvCRFs assume that pair of pixels $i,j$ are joint independent if Manhattan distance $d(i,j)>k$ (You may regard $k$ as the filter size)
- Others word, $\sum_{i\neq j\leq N}\psi_p(x_i,x_j)=0$ if $d(i,j)>k$
- reformulate message passing function to truncated Gaussian kernel

### Kernel function

- consider input $P$ with shape $[bs,c,h,w]$, the Gaussian kernel $g$ defined by feature vectors $f_1,\cdots,f_d$ each with shape $[bs,h,w]$
- $k_g[bs,dx,dy,x,y]=\\ \exp(-\sum_{i=1}^d\frac{|f_i^{(d)}[bs,x,y]-f^{(d)}_i[bs,x-dx,y-dy]|^2}{2\theta_i^2})$
- where $\theta_i$ are learnable parameters
- Merge gaussian kernel $K=\sum_{i=1}^sw_ig_i$
- In practice, $f_i$ is choose to be the RGB intensity and spatial location $x,y$ ($s=2$)



Note: The animation evaluation of ConvCRFs can be founded at https://dongdong-zoez.github.io/sources/ConvCRFs/index.html

### Dataset

以下網站是大神提供的 [SBD](https://www.sun11.me/blog/2018/how-to-use-10582-trainaug-images-on-DeeplabV3-code/) 資料集，trainaug.txt 可以從 [這](https://gist.githubusercontent.com/sun11/2dbda6b31acc7c6292d14a872d0c90b7/raw/5f5a5270089239ef2f6b65b1cc55208355b5acca/trainaug.txt) 拿到，SBD 是 Segmentation 領域最常用來衡量模型表現的資料，算是 Pascal VOC 的數據增廣版本

我們會用到以下資料

- train: SBD dataset (Pascal VOC dataset) with 10582 images (trainaug)
- val  : Pascal VOC 2012 val with 1449 images

以下專案提供了 [ConvCRFs](https://arxiv.org/pdf/1805.04777.pdf) 的訓練代碼

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/MyDrive')
!git clone https://github.com/DongDong-Zoez/ConvCRFs-training.git

fatal: destination path 'ConvCRFs-training' already exists and is not an empty directory.


In [ ]:
os.chdir('/content/gdrive/MyDrive')
!python /content/gdrive/MyDrive/ConvCRFs-training/main.py --data-path '/content/gdrive/MyDrive' -b 1 --epochs 1

device: cuda
batch size: 1
num workers: 0
learning rate: 5e-05
loss function: CrossEntropy
optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 5e-05
    lr: 5e-05
    maximize: False
    weight_decay: 0.0005
)
scaler: None
epochs: 1
weight decay: 0.0005
lr_scheduler: PolynomialScheduler
start epochs: 0
aux: False
crf: False
fullscaleFeat: None
held out images: 10
train with held out: True
num train images: 10
num val images: 1449
num steps per epoch: 10
train transfrom: [RandomResize, RandomCrop, RandomHorizontalFilp, RandomRotate, ToTensor, Normalize]
val transforms: [Resize, ToTensor, Normalize]

Epoch: [0]  [ 0/10]  eta: 0:00:20  lr: 0.000045  loss: 3.3752 (3.3752)  time: 2.0921  data: 0.0355  max mem: 2069
Epoch: [0] Total time: 0:00:05
Test:  [  0/363]  eta: 0:02:48    time: 0.4655  data: 0.0811  max mem: 2671
Test:  [100/363]  eta: 0:02:01    time: 0.4770  data: 0.0685  max mem: 2671